In [ ]:
{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# Tecana Technical Analysis Library Demo\n",
    "\n",
    "This notebook demonstrates how to use the Tecana library for technical analysis of financial market data."
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Installation\n",
    "\n",
    "First, let's install the library using pip:"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "!pip install tecana"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Import Libraries"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "import tecana as tec\n",
    "import pandas as pd\n",
    "import numpy as np\n",
    "import matplotlib.pyplot as plt\n",
    "\n",
    "# Set up plotting\n",
    "%matplotlib inline\n",
    "plt.style.use('seaborn-v0_8-darkgrid')\n",
    "plt.rcParams['figure.figsize'] = (14, 8)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Generate Sample OHLCV Data\n",
    "\n",
    "Let's create a realistic OHLCV (Open-High-Low-Close-Volume) dataset with random price movements:"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "def generate_ohlc_data(days=200, start_price=100.00):\n",
    "    np.random.seed(42)  # For reproducibility\n",
    "    \n",
    "    # Generate dates\n",
    "    dates = pd.date_range(end=pd.Timestamp.now().normalize(), periods=days)\n",
    "    \n",
    "    # Initial price\n",
    "    price = start_price\n",
    "    \n",
    "    # Lists to store OHLCV data\n",
    "    opens = []\n",
    "    highs = []\n",
    "    lows = []\n",
    "    closes = []\n",
    "    volumes = []\n",
    "    \n",
    "    # Generate price movement with some trend and volatility\n",
    "    for i in range(days):\n",
    "        # Daily volatility (percentage)\n",
    "        volatility = max(0.5, min(5.0, 2.0 + 0.5 * np.sin(i/20)))\n",
    "        \n",
    "        # Daily trend component (can create periods of up/down trends)\n",
    "        trend = 0.1 * np.sin(i/40) + 0.05 * np.sin(i/20)\n",
    "        \n",
    "        # Generate open price with some gap from previous close\n",
    "        if i == 0:\n",
    "            open_price = price\n",
    "        else:\n",
    "            # Possible gap up/down on open (±1%)\n",
    "            gap = np.random.normal(0, 0.01)\n",
    "            open_price = closes[-1] * (1 + gap)\n",
    "        \n",
    "        # Generate daily range and close\n",
    "        daily_change = np.random.normal(trend, volatility/100)\n",
    "        close_price = open_price * (1 + daily_change)\n",
    "        \n",
    "        # Generate high and low with realistic relationships\n",
    "        price_range = max(open_price, close_price) * (volatility/100)\n",
    "        high_price = max(open_price, close_price) + price_range * np.random.uniform(0.2, 1.0)\n",
    "        low_price = min(open_price, close_price) - price_range * np.random.uniform(0.2, 1.0)\n",
    "        \n",
    "        # Generate volume (higher on larger price moves)\n",
    "        base_volume = 100000 + 50000 * np.sin(i/30)  # Cyclical volume pattern\n",
    "        volume = int(base_volume * (1 + 2 * abs(daily_change)))\n",
    "        \n",
    "        # Round prices to 2 decimal places\n",
    "        open_price = round(open_price, 2)\n",
    "        high_price = round(high_price, 2)\n",
    "        low_price = round(low_price, 2)\n",
    "        close_price = round(close_price, 2)\n",
    "        \n",
    "        # Store values\n",
    "        opens.append(open_price)\n",
    "        highs.append(high_price)\n",
    "        lows.append(low_price)\n",
    "        closes.append(close_price)\n",
    "        volumes.append(volume)\n",
    "        \n",
    "        # Update price for trend continuity\n",
    "        price = close_price\n",
    "        \n",
    "    # Create DataFrame\n",
    "    df = pd.DataFrame({\n",
    "        'open': opens,\n",
    "        'high': highs,\n",
    "        'low': lows,\n",
    "        'close': closes,\n",
    "        'volume': volumes\n",
    "    }, index=dates)\n",
    "    \n",
    "    return df\n",
    "\n",
    "# Generate our test data\n",
    "ohlcv_data = generate_ohlc_data(days=200, start_price=100.00)\n",
    "\n",
    "# Display the first few rows\n",
    "print(f\"Generated {len(ohlcv_data)} days of OHLCV data\")\n",
    "ohlcv_data.head()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "Let's plot our price data to verify it looks realistic:"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "plt.figure(figsize=(15, 7))\n",
    "plt.plot(ohlcv_data.index, ohlcv_data['close'], label='Close Price')\n",
    "plt.title('Sample Price Data')\n",
    "plt.xlabel('Date')\n",
    "plt.ylabel('Price')\n",
    "plt.legend()\n",
    "plt.grid(True)\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Basic Technical Indicators\n",
    "\n",
    "Let's calculate some basic technical indicators using the Tecana library:"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Calculate RSI (Relative Strength Index)\n",
    "df_rsi = tec.rsi(ohlcv_data, window=14)\n",
    "print(\"RSI added. New columns:\", [col for col in df_rsi.columns if col not in ohlcv_data.columns])\n",
    "\n",
    "# Calculate MACD (Moving Average Convergence Divergence)\n",
    "df_macd = tec.macd(df_rsi)\n",
    "print(\"MACD added. New columns:\", [col for col in df_macd.columns if col not in df_rsi.columns])\n",
    "\n",
    "# Calculate Bollinger Bands\n",
    "df_bb = tec.bb(df_macd, window=20)\n",
    "print(\"Bollinger Bands added. New columns:\", [col for col in df_bb.columns if col not in df_macd.columns])\n",
    "\n",
    "# Calculate Average True Range\n",
    "df_atr = tec.atr(df_bb, window=14)\n",
    "print(\"ATR added. New columns:\", [col for col in df_atr.columns if col not in df_bb.columns])"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "Let's visualize a couple of these indicators:"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(15, 10), gridspec_kw={'height_ratios': [3, 1]})\n",
    "\n",
    "# Price and Bollinger Bands\n",
    "ax1.plot(df_atr.index, df_atr['close'], label='Close Price')\n",
    "ax1.plot(df_atr.index, df_atr['bbh'], 'r--', label='Upper BB')\n",
    "ax1.plot(df_atr.index, df_atr['bbl'], 'g--', label='Lower BB')\n",
    "ax1.fill_between(df_atr.index, df_atr['bbh'], df_atr['bbl'], alpha=0.1, color='gray')\n",
    "ax1.set_title('Price with Bollinger Bands')\n",
    "ax1.set_ylabel('Price')\n",
    "ax1.legend()\n",
    "ax1.grid(True)\n",
    "\n",
    "# RSI\n",
    "ax2.plot(df_atr.index, df_atr['rsi'], label='RSI')\n",
    "ax2.axhline(y=70, color='r', linestyle='-', alpha=0.3)\n",
    "ax2.axhline(y=30, color='g', linestyle='-', alpha=0.3)\n",
    "ax2.fill_between(df_atr.index, 70, 30, alpha=0.1, color='gray')\n",
    "ax2.set_title('RSI (14)')\n",
    "ax2.set_ylabel('RSI')\n",
    "ax2.set_ylim(0, 100)\n",
    "ax2.legend()\n",
    "ax2.grid(True)\n",
    "\n",
    "plt.tight_layout()\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Generate Trading Signals\n",
    "\n",
    "Now, let's calculate some trading signals using the indicators we've generated.\n",
    "For signal interpretation:\n",
    "- '-1' indicates a BUY signal (bullish)\n",
    "- '1' indicates a SELL signal (bearish)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Calculate trend signals\n",
    "df_signals = tec.macd_t(df_atr.copy())  # MACD trend signal\n",
    "df_signals = tec.rsi_z(df_signals)      # RSI zone signal\n",
    "df_signals = tec.bb_t(df_signals)       # Bollinger Bands trend signal\n",
    "\n",
    "# Calculate volatility signal\n",
    "df_signals = tec.atr_v(df_signals, threshold=1.5)  # ATR volatility signal\n",
    "\n",
    "# Display the signals\n",
    "signal_cols = ['macd_t', 'rsi_z', 'bb_t', 'atr_v']\n",
    "print(\"Signal summary for last 10 days:\")\n",
    "df_signals[signal_cols].tail(10)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### Explanation of Signals\n",
    "\n",
    "- **macd_t**: MACD trend signal\n",
    "  - -1: Buy signal when MACD histogram is positive\n",
    "  - 1: Sell signal when MACD histogram is negative\n",
    "\n",
    "- **rsi_z**: RSI zone signal (overbought/oversold)\n",
    "  - -1: Buy signal when RSI is in oversold zone\n",
    "  - 1: Sell signal when RSI is in overbought zone\n",
    "\n",
    "- **bb_t**: Bollinger Bands trend signal\n",
    "  - -1: Buy signal when price touches lower band with positive momentum\n",
    "  - 1: Sell signal when price touches upper band with negative momentum\n",
    "  \n",
    "- **atr_v**: ATR volatility signal (not a buy/sell signal)\n",
    "  - 1: Indicates high volatility (ATR above threshold)\n",
    "  - NaN: Normal volatility"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Using the Custom Method\n",
    "\n",
    "The `custom` method allows applying multiple indicators and signals in a single call."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Use the custom method to apply multiple indicators at once\n",
    "custom_df = tec.custom(ohlcv_data,\n",
    "    ['rsi', 14],                  # RSI with window=14\n",
    "    ['macd'],                     # MACD with default parameters\n",
    "    ['bb', 20, 2],                # Bollinger Bands with window=20 and std=2\n",
    "    ['atr', {'window': 14}]       # ATR with window=14 (using kwargs)\n",
    ")\n",
    "\n",
    "# Add multiple signals using custom method\n",
    "custom_signals = tec.custom(custom_df,\n",
    "    ['rsi_z'],                    # RSI zone signal (using default parameters)\n",
    "    ['macd_t'],                   # MACD trend signal\n",
    "    ['bb_t', 14],                 # BB trend signal with custom parameter\n",
    "    ['atr_v', 1.2]                # ATR volatility signal with custom threshold\n",
    ")\n",
    "\n",
    "print(\"\\nIndicators added:\")\n",
    "print([col for col in custom_df.columns if col not in ohlcv_data.columns])\n",
    "\n",
    "print(\"\\nSignals added:\")\n",
    "print([col for col in custom_signals.columns if col not in custom_df.columns])\n",
    "\n",
    "# View signal values\n",
    "signal_cols = ['rsi_z', 'macd_t', 'bb_t', 'atr_v']\n",
    "print(\"\\nSignals for the last 5 days:\")\n",
    "custom_signals[signal_cols].tail()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Visualize Signals On Price Chart\n",
    "\n",
    "Let's visualize buy and sell signals on a price chart:"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Get buy and sell signals from MACD trend\n",
    "buy_signals = custom_signals[custom_signals['macd_t'] == -1].index\n",
    "sell_signals = custom_signals[custom_signals['macd_t'] == 1].index\n",
    "\n",
    "# Plot price with signals\n",
    "plt.figure(figsize=(15, 7))\n",
    "plt.plot(custom_signals.index, custom_signals['close'], label='Close Price', alpha=0.8)\n",
    "\n",
    "# Plot buy signals (green arrows)\n",
    "plt.scatter(buy_signals, custom_signals.loc[buy_signals, 'close'], \n",
    "            marker='^', color='green', s=100, label='Buy Signal', alpha=0.8)\n",
    "\n",
    "# Plot sell signals (red arrows)\n",
    "plt.scatter(sell_signals, custom_signals.loc[sell_signals, 'close'], \n",
    "            marker='v', color='red', s=100, label='Sell Signal', alpha=0.8)\n",
    "\n",
    "plt.title('Price Chart with MACD Buy/Sell Signals')\n",
    "plt.xlabel('Date')\n",
    "plt.ylabel('Price')\n",
    "plt.legend()\n",
    "plt.grid(True)\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Combined Signal Strategy\n",
    "\n",
    "Let's create a simple strategy that combines multiple signals:"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Create a combined signal (Buy when both MACD and RSI agree)\n",
    "custom_signals['combined_signal'] = 0\n",
    "\n",
    "# Strong buy signal when both MACD and RSI suggest buying\n",
    "custom_signals.loc[(custom_signals['macd_t'] == -1) & \n",
    "                   (custom_signals['rsi_z'] == -1), 'combined_signal'] = -1\n",
    "\n",
    "# Strong sell signal when both MACD and RSI suggest selling\n",
    "custom_signals.loc[(custom_signals['macd_t'] == 1) & \n",
    "                   (custom_signals['rsi_z'] == 1), 'combined_signal'] = 1\n",
    "\n",
    "# Count the number of buy and sell signals\n",
    "buy_count = (custom_signals['combined_signal'] == -1).sum()\n",
    "sell_count = (custom_signals['combined_signal'] == 1).sum()\n",
    "\n",
    "print(f\"Combined strategy generated {buy_count} buy signals and {sell_count} sell signals\")\n",
    "\n",
    "# Show the latest signals\n",
    "last_days = 15\n",
    "print(f\"\\nSignals for the last {last_days} days:\")\n",
    "custom_signals[['close', 'macd_t', 'rsi_z', 'combined_signal']].tail(last_days)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Conclusion\n",
    "\n",
    "In this demo, we've shown how to:\n",
    "\n",
    "1. Install and import the Tecana library\n",
    "2. Generate sample OHLCV data\n",
    "3. Calculate various technical indicators\n",
    "4. Generate trading signals\n",
    "5. Use the powerful `custom` method to apply multiple indicators and signals at once\n",
    "6. Create a simple combined trading strategy\n",
    "\n",
    "The Tecana library offers a comprehensive set of technical indicators and signals with a clean, easy-to-use API. The `custom` method makes it particularly efficient to work with multiple indicators simultaneously."
   ]
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.8.10"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 4
}